In [1]:
! sudo apt-get update
! sudo mkdir -p /usr/share/man/man1
! sudo apt-get install -y openjdk-11-jdk
! pip install pyspark

Get:1 http://deb.debian.org/debian buster InRelease [122 kB]
Get:2 http://deb.debian.org/debian-security buster/updates InRelease [34.8 kB]
Get:3 http://deb.debian.org/debian buster-updates InRelease [56.6 kB]
Get:4 http://deb.debian.org/debian buster/main amd64 Packages [7,909 kB]
Get:5 http://deb.debian.org/debian-security buster/updates/main amd64 Packages [480 kB]
Get:6 http://deb.debian.org/debian buster-updates/main amd64 Packages [8,788 B]
Fetched 8,611 kB in 2s (3,517 kB/s)




The following additional packages will be installed:
  at-spi2-core ca-certificates-java dbus dbus-user-session
  dconf-gsettings-backend dconf-service dmsetup fonts-dejavu-extra
  glib-networking glib-networking-common glib-networking-services
  gsettings-desktop-schemas java-common libapparmor1 libargon2-1 libasound2
  libasound2-data libatk-bridge2.0-0 libatk-wrapper-java
  libatk-wrapper-java-jni libatspi2.0-0 libcap2 libcolord2 libcryptsetup12
  libdconf1 libdevmapper1.02.1 libdrm-amdgpu1 libdrm-com

In [2]:
from pyspark import SparkContext

In [3]:
# Crear un contexto
sc = SparkContext(master='local', appName='trasform_action')

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/04/25 11:06:30 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
# crear rdds
rdd1 = sc.parallelize([1, 2, 3])
type(rdd1)

pyspark.rdd.RDD

In [5]:
# ver la salida
rdd1.collect()

[1, 2, 3]

In [6]:
sc

<SparkContext master=local appName=trasform_action>

In [7]:
!ls /data/

deporte.csv	 deportistaError.csv  modelo_relacional.jpg
deportista2.csv  evento.csv	      paises.csv
deportista.csv	 juegos.csv	      resultados.csv


In [8]:
olympics_teams_rdd = sc.textFile('/data/paises.csv').map(
    lambda line: line.split(',')
)

In [9]:
# hasta que no se realice una accion no se puede ver los errores en los rdds
olympics_teams_rdd.take(5)

[['id', 'equipo', 'sigla'],
 ['1', '30. Februar', 'AUT'],
 ['2', 'A North American Team', 'MEX'],
 ['3', 'Acipactli', 'MEX'],
 ['4', 'Acturus', 'ARG']]

In [10]:
# (x[2]) -> tomar los valores de las siglas que estan el posc 2 y comvertirlo en una tupla
# .distinct() eliminar duplicados
# .count() # contar
olympics_teams_rdd.map(lambda x: (x[2])).distinct().count()

231

In [11]:
# agrupar los equipos por pais y mostrar los primeros 5
olympics_teams_rdd.map(lambda x: (x[2], x[1])).groupByKey().mapValues(len).take(5)

[('sigla', 1), ('AUT', 11), ('MEX', 9), ('ARG', 18), ('AFG', 1)]

In [12]:
# agrupar los equipos por lista y mostar los equpos en una lista 
olympics_teams_rdd.map(lambda x: (x[2], x[1])).groupByKey().mapValues(list).take(5)

[('sigla', ['equipo']),
 ('AUT',
  ['30. Februar',
   'Austria',
   'Austria-1',
   'Austria-2',
   'Breslau',
   'Brigantia',
   'Donar III',
   'Evita VI',
   'May-Be 1960',
   '"R.-V. Germania; Leitmeritz"',
   'Surprise']),
 ('MEX',
  ['A North American Team',
   'Acipactli',
   'Chamukina',
   'Mexico',
   'Mexico-1',
   'Mexico-2',
   'Nausikaa 4',
   'Tlaloc',
   'Xolotl']),
 ('ARG',
  ['Acturus',
   'Antares',
   'Arcturus',
   'Ardilla',
   'Argentina',
   'Argentina-1',
   'Argentina-2',
   'Blue Red',
   'Covunco III',
   'Cupidon III',
   'Djinn',
   'Gullvinge',
   'Matrero II',
   'Mizar',
   'Pampero',
   'Rampage',
   'Tango',
   'Wiking']),
 ('AFG', ['Afghanistan'])]

In [13]:
# rdd de los equipos argentinos
argentinian_team = olympics_teams_rdd.filter(lambda l: 'ARG' in l)
argentinian_team.take(5)

[['4', 'Acturus', 'ARG'],
 ['37', 'Antares', 'ARG'],
 ['42', 'Arcturus', 'ARG'],
 ['43', 'Ardilla', 'ARG'],
 ['45', 'Argentina', 'ARG']]

In [14]:
# contar hasta la cantindad de milisegindos que se indique como parametro
argentinian_team.countApprox(20)

18

In [15]:
olympics_athletes_rdd1 = sc.textFile('/data/deportista.csv').map(
    lambda line: line.split(',')
)

olympics_athletes_rdd2 = sc.textFile('/data/deportista2.csv').map(
    lambda line: line.split(',')
)

In [16]:
# union de los dos rdds
olympics_athletes_rdd = olympics_athletes_rdd1.union(olympics_athletes_rdd2)

In [17]:
olympics_athletes_rdd.count()

135572

In [18]:
olympics_teams_rdd.top(2)

[['id', 'equipo', 'sigla'], ['999', 'Stella-2', 'NOR']]

In [19]:
olympics_athletes_rdd.top(2)

[['deportista_id', 'nombre', 'genero', 'edad', 'altura', 'peso', 'equipo_id'],
 ['99999', 'Alexander Grant Alick Rennie', '1', '32', '182', '71', '967']]

In [52]:
countries_athletes_rdd = olympics_athletes_rdd.map(
    lambda l: [l[-1], l[:-1]]).join(olympics_teams_rdd.map(
        lambda x: [x[0], x[2]]))

countries_athletes.takeSample(False, 6, 25)

[('362', (['131505', 'Steven Woodburn', '1', '24', '185', '90'], 'FRA')),
 ('967', (['13626', 'Jill Brresen', '2', '22', '170', '57'], 'RSA')),
 ('482', (['44299', 'Gumundur Gumundsson', '1', '23', '174', '77'], 'ISL')),
 ('970', (['68062', 'Lee MinHui', '2', '28', '174', '65'], 'KOR')),
 ('794', (['92442', 'Luis Paz Zoldan', '1', '19', '187', '82'], 'PER')),
 ('413', (['26822', 'Jared Mark Deacon', '1', '24', '185', '77'], 'GBR'))]

In [56]:
countries_athletes_rdd.map(lambda x: x[1][0]).filter(lambda l: 'Alexander Grant Alick Rennie' in l[1]).take(2)

[['99999', 'Alexander Grant Alick Rennie', '1', '32', '182', '71']]

In [27]:
result_rdd = sc.textFile('/data/resultados.csv').map(
    lambda line: line.split(',')
)

In [28]:
result_winner_rdd = result_rdd.filter(lambda l: 'NA' not in l[1])
result_winner_rdd.take(2)

[['resultado_id', 'medalla', 'deportista_id', 'juego_id', 'evento_id'],
 ['4', 'Gold', '4', '2', '4']]

In [57]:
countries_athletes.map(
    lambda l: [l[1][0][0], l[1][1]]).join(result_winner_rdd.map(
        lambda x: [x[2], x[1]])).takeSample(False, 6, 25)

[('54832', ('ESP', 'Silver')),
 ('135219', ('SLO', 'Bronze')),
 ('106724', ('NED', 'Gold')),
 ('112317', ('USA', 'Gold')),
 ('124518', ('NED', 'Gold')),
 ('42668', ('USA', 'Bronze'))]

# Operaciones numéricas

In [59]:
values_medals = {'Gold': 7, 'Silver': 5, 'Bronze': 4}


In [76]:
country_medals_rdd = countries_athletes.map(
    lambda l: [l[1][0][0], l[1][1]]).join(result_winner_rdd.map(
        lambda x: [x[2], x[1]])).map(
            lambda x: [x[1][0], x[1][1]]
        )

In [77]:
country_medals_rdd.take(5)

[['CHN', 'Silver'],
 ['CHN', 'Bronze'],
 ['CHN', 'Silver'],
 ['CHN', 'Silver'],
 ['CHN', 'Silver']]

In [86]:
country_medals_rdd = country_medals_rdd.map(lambda r: (r[0], values_medals[r[1]]))
country_medals_rdd.take(5)

[('CHN', 5), ('CHN', 4), ('CHN', 5), ('CHN', 5), ('CHN', 5)]

In [87]:
from operator import add

In [89]:
result_final = country_medals_rdd.reduceByKey((add)).sortBy(lambda x: x[1], ascending=False)

In [90]:
result_final.take(10)

[('USA', 32137),
 ('URS', 14834),
 ('GBR', 10925),
 ('GER', 10896),
 ('FRA', 9265),
 ('ITA', 8755),
 ('SWE', 8110),
 ('CAN', 7209),
 ('AUS', 6755),
 ('HUN', 6142)]

In [ ]:
sc.stop()